In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, col


In [2]:
spark = SparkSession.builder  \
    .appName('python_etl') \
    .getOrCreate()

24/08/29 15:07:29 WARN Utils: Your hostname, basel resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlp3s0)
24/08/29 15:07:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/29 15:07:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df  = spark.read    \
    .format('csv')  \
    .option('sep',';')\
    .option('header','true')\
    .option('inferschema','true') \
    .load('../../data/raw/vaccination-data.csv')
df.show()

+--------------------+----+----------+-----------+------------+------------------+-----------------------------+-------------------------+------------------------------------+-----------------+------------------------+-------------+------------------+--------------------------+------------------------+-------------------------------+----+
|             COUNTRY|ISO3|WHO_REGION|DATA_SOURCE|DATE_UPDATED|TOTAL_VACCINATIONS|PERSONS_VACCINATED_1PLUS_DOSE|TOTAL_VACCINATIONS_PER100|PERSONS_VACCINATED_1PLUS_DOSE_PER100|PERSONS_LAST_DOSE|PERSONS_LAST_DOSE_PER100|VACCINES_USED|FIRST_VACCINE_DATE|NUMBER_VACCINES_TYPES_USED|PERSONS_BOOSTER_ADD_DOSE|PERSONS_BOOSTER_ADD_DOSE_PER100|_c16|
+--------------------+----+----------+-----------+------------+------------------+-----------------------------+-------------------------+------------------------------------+-----------------+------------------------+-------------+------------------+--------------------------+------------------------+---------------

24/08/29 15:07:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: COUNTRY, ISO3, WHO_REGION, DATA_SOURCE, DATE_UPDATED, TOTAL_VACCINATIONS, PERSONS_VACCINATED_1PLUS_DOSE, TOTAL_VACCINATIONS_PER100, PERSONS_VACCINATED_1PLUS_DOSE_PER100, PERSONS_LAST_DOSE, PERSONS_LAST_DOSE_PER100, VACCINES_USED, FIRST_VACCINE_DATE, NUMBER_VACCINES_TYPES_USED, PERSONS_BOOSTER_ADD_DOSE, PERSONS_BOOSTER_ADD_DOSE_PER100, 
 Schema: COUNTRY, ISO3, WHO_REGION, DATA_SOURCE, DATE_UPDATED, TOTAL_VACCINATIONS, PERSONS_VACCINATED_1PLUS_DOSE, TOTAL_VACCINATIONS_PER100, PERSONS_VACCINATED_1PLUS_DOSE_PER100, PERSONS_LAST_DOSE, PERSONS_LAST_DOSE_PER100, VACCINES_USED, FIRST_VACCINE_DATE, NUMBER_VACCINES_TYPES_USED, PERSONS_BOOSTER_ADD_DOSE, PERSONS_BOOSTER_ADD_DOSE_PER100, _c16
Expected: _c16 but found: 
CSV file: file:///home/basel/basel/python_Etl/data/raw/vaccination-data.csv


In [4]:
# dropping unneccessary columns
modified_df = df.drop('ISO3','DATA_SOURCE','DATA_SOURCE','NUMBER_VACCINES_TYPES_USED','_c16','VACCINES_USED')


In [5]:
# replacing nulls with values
mean_value = modified_df.select(mean(col('TOTAL_VACCINATIONS_PER100'))).collect()[0][0]
df_filled = modified_df.fillna(value=mean_value,subset='TOTAL_VACCINATIONS_PER100')
df_filled.show()

+--------------------+----------+------------+------------------+-----------------------------+-------------------------+------------------------------------+-----------------+------------------------+------------------+------------------------+-------------------------------+
|             COUNTRY|WHO_REGION|DATE_UPDATED|TOTAL_VACCINATIONS|PERSONS_VACCINATED_1PLUS_DOSE|TOTAL_VACCINATIONS_PER100|PERSONS_VACCINATED_1PLUS_DOSE_PER100|PERSONS_LAST_DOSE|PERSONS_LAST_DOSE_PER100|FIRST_VACCINE_DATE|PERSONS_BOOSTER_ADD_DOSE|PERSONS_BOOSTER_ADD_DOSE_PER100|
+--------------------+----------+------------+------------------+-----------------------------+-------------------------+------------------------------------+-----------------+------------------------+------------------+------------------------+-------------------------------+
|               Aruba|      AMRO|  29/12/2023|            217124|                      90493.0|                      203|                                  85|        

In [6]:
# droping row that has null values 
no_nulldf= df_filled.dropna()
no_nulldf.show()

+--------------------+----------+------------+------------------+-----------------------------+-------------------------+------------------------------------+-----------------+------------------------+------------------+------------------------+-------------------------------+
|             COUNTRY|WHO_REGION|DATE_UPDATED|TOTAL_VACCINATIONS|PERSONS_VACCINATED_1PLUS_DOSE|TOTAL_VACCINATIONS_PER100|PERSONS_VACCINATED_1PLUS_DOSE_PER100|PERSONS_LAST_DOSE|PERSONS_LAST_DOSE_PER100|FIRST_VACCINE_DATE|PERSONS_BOOSTER_ADD_DOSE|PERSONS_BOOSTER_ADD_DOSE_PER100|
+--------------------+----------+------------+------------------+-----------------------------+-------------------------+------------------------------------+-----------------+------------------------+------------------+------------------------+-------------------------------+
|               Aruba|      AMRO|  29/12/2023|            217124|                      90493.0|                      203|                                  85|        

In [10]:
#writing data into csv 
no_nulldf.write.csv('../../data/transformed/data1.csv')
